In [ ]:
import json
import os
from typing import Dict, List, Tuple, Optional
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from PIL import Image
import numpy as np


class PolygonColorDataset(Dataset):
    def __init__(self,
                 data_json_path: str,
                 input_dir: str,
                 output_dir: str,
                 image_size: int = 256,
                 augmentations: Optional[transforms.Compose] = None,
                 color_to_idx: Optional[Dict[str, int]] = None):

        self.input_dir = input_dir
        self.output_dir = output_dir
        self.image_size = image_size
        self.augmentations = augmentations

        # Load data from JSON
        with open(data_json_path, 'r') as f:
            self.data = json.load(f)

        # Create color mappings
        if color_to_idx:
            self.color_to_idx = color_to_idx
            self.colors = sorted(list(self.color_to_idx.keys()))
        else:
            self.colors = sorted(list(set(item['colour'] for item in self.data)))
            self.color_to_idx = {color: idx for idx, color in enumerate(self.colors)}

        self.num_colors = len(self.colors)

        print(f"Dataset initialized with {len(self.data)} samples")
        print(f"Colors: {self.colors}")
        print(f"Color mapping: {self.color_to_idx}")

        # Define transforms
        self.transform = transforms.Compose([
            transforms.Resize((image_size, image_size)),
            transforms.ToTensor(),
        ])

        # Normalize to [-1, 1] for better training stability
        self.normalize = transforms.Normalize([0.5], [0.5])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]

        # Load input polygon image
        input_path = os.path.join(self.input_dir, item['input_polygon'])
        input_image = Image.open(input_path).convert('RGB')

        # Load target output image
        output_path = os.path.join(self.output_dir, item['output_image'])
        output_image = Image.open(output_path).convert('RGB')

        # Apply transforms
        input_tensor = self.transform(input_image)
        output_tensor = self.transform(output_image)

        # Normalize to [-1, 1]
        input_tensor = self.normalize(input_tensor)
        output_tensor = self.normalize(output_tensor)

        # Get color information
        color_name = item['colour']
        color_idx = self.color_to_idx[color_name]

        # Create one-hot encoding for color
        color_onehot = torch.zeros(self.num_colors)
        color_onehot[color_idx] = 1.0

        # Apply augmentations if provided
        if self.augmentations:
            # Apply same augmentation to both input and output
            seed = np.random.randint(2147483647)  # make a seed with numpy generator

            # Apply to input
            torch.manual_seed(seed)
            input_tensor = self.augmentations(input_tensor)

            # Apply to output
            torch.manual_seed(seed)
            output_tensor = self.augmentations(output_tensor)

        return {
            'input_image': input_tensor,
            'color_onehot': color_onehot,
            'color_idx': color_idx,
            'color_name': color_name,
            'output_image': output_tensor,
            'input_filename': item['input_polygon'],
            'output_filename': item['output_image']
        }

    def get_color_embedding_dim(self):
        """Return the dimension of color embeddings (number of unique colors)"""
        return self.num_colors


def get_data_loaders(train_json_path: str,
                    val_json_path: str,
                    train_input_dir: str,
                    train_output_dir: str,
                    val_input_dir: str,
                    val_output_dir: str,
                    batch_size: int = 16,
                    image_size: int = 256,
                    num_workers: int = 4,
                    use_augmentations: bool = True):

    # Define augmentations for training
    train_augmentations = None
    if use_augmentations:
        train_augmentations = transforms.Compose([
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomRotation(degrees=15),
            transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.05),
        ])

    # Create datasets
    train_dataset = PolygonColorDataset(
        data_json_path=train_json_path,
        input_dir=train_input_dir,
        output_dir=train_output_dir,
        image_size=image_size,
        augmentations=train_augmentations
    )

    # Use the color mapping from the training set for the validation set
    color_to_idx = train_dataset.color_to_idx

    val_dataset = PolygonColorDataset(
        data_json_path=val_json_path,
        input_dir=val_input_dir,
        output_dir=val_output_dir,
        image_size=image_size,
        augmentations=None,  # No augmentations for validation
        color_to_idx=color_to_idx
    )

    # Create data loaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True,
        drop_last=True
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True,
        drop_last=False
    )

    # Return color information for model initialization
    color_info = {
        'colors': train_dataset.colors,
        'color_to_idx': train_dataset.color_to_idx,
        'num_colors': train_dataset.num_colors
    }

    return train_loader, val_loader, color_info


# Example usage and testing
if __name__ == "__main__":
    # Test dataset loading
    dataset = PolygonColorDataset(
        data_json_path="/content/drive/MyDrive/dataset/dataset/training/data.json",
        input_dir="/content/drive/MyDrive/dataset/dataset/training/inputs",
        output_dir="/content/drive/MyDrive/dataset/dataset/training/outputs",
        image_size=256
    )

    # Test a single sample
    sample = dataset[0]
    print(f"Sample keys: {sample.keys()}")
    print(f"Input image shape: {sample['input_image'].shape}")
    print(f"Output image shape: {sample['output_image'].shape}")
    print(f"Color onehot shape: {sample['color_onehot'].shape}")
    print(f"Color name: {sample['color_name']}")
    print(f"Color index: {sample['color_idx']}")

    # Test data loader
    from torch.utils.data import DataLoader
    loader = DataLoader(dataset, batch_size=4, shuffle=True)
    batch = next(iter(loader))
    print(f"\nBatch input shape: {batch['input_image'].shape}")
    print(f"Batch output shape: {batch['output_image'].shape}")
    print(f"Batch color onehot shape: {batch['color_onehot'].shape}")

Dataset initialized with 56 samples
Colors: ['blue', 'cyan', 'green', 'magenta', 'orange', 'purple', 'red', 'yellow']
Color mapping: {'blue': 0, 'cyan': 1, 'green': 2, 'magenta': 3, 'orange': 4, 'purple': 5, 'red': 6, 'yellow': 7}
Sample keys: dict_keys(['input_image', 'color_onehot', 'color_idx', 'color_name', 'output_image', 'input_filename', 'output_filename'])
Input image shape: torch.Size([3, 256, 256])
Output image shape: torch.Size([3, 256, 256])
Color onehot shape: torch.Size([8])
Color name: cyan
Color index: 1

Batch input shape: torch.Size([4, 3, 256, 256])
Batch output shape: torch.Size([4, 3, 256, 256])
Batch color onehot shape: torch.Size([4, 8])


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import Optional


class DoubleConv(nn.Module):
    """(convolution => [BN] => ReLU) * 2"""

    def __init__(self, in_channels: int, out_channels: int, mid_channels: Optional[int] = None):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)


class Down(nn.Module):
    """Downscaling with maxpool then double conv"""

    def __init__(self, in_channels: int, out_channels: int):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)


class Up(nn.Module):
    """Upscaling then double conv"""

    def __init__(self, in_channels: int, out_channels: int, bilinear: bool = True):
        super().__init__()

        # if bilinear, use the normal convolutions to reduce the number of channels
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
            self.conv = DoubleConv(in_channels, out_channels, in_channels // 2)
        else:
            self.up = nn.ConvTranspose2d(in_channels, in_channels // 2, kernel_size=2, stride=2)
            self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        # input is CHW
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])
        # if you have padding issues, see
        # https://github.com/HaiyongJiang/U-Net-Pytorch-Unstructured-Buggy/commit/0e854509c2cea854e247a9c615f175f76fbb2e3a
        # https://github.com/xiaopeng-liao/Pytorch-UNet/commit/8ebac70e633bac59fc22bb5195e513d5832fb3bd
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)


class OutConv(nn.Module):
    def __init__(self, in_channels: int, out_channels: int):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)


class FiLM(nn.Module):
    """Feature-wise Linear Modulation layer"""

    def __init__(self, condition_dim: int, feature_dim: int):
        super().__init__()
        self.condition_dim = condition_dim
        self.feature_dim = feature_dim

        # Linear layers to generate scale (gamma) and shift (beta) parameters
        self.gamma_linear = nn.Linear(condition_dim, feature_dim)
        self.beta_linear = nn.Linear(condition_dim, feature_dim)

    def forward(self, x, condition):
        """
        Args:
            x: Feature tensor of shape (B, C, H, W)
            condition: Condition tensor of shape (B, condition_dim)

        Returns:
            Modulated features of shape (B, C, H, W)
        """
        # Generate scale and shift parameters
        gamma = self.gamma_linear(condition)  # (B, feature_dim)
        beta = self.beta_linear(condition)    # (B, feature_dim)

        # Reshape for broadcasting with feature maps
        gamma = gamma.view(gamma.size(0), gamma.size(1), 1, 1)  # (B, C, 1, 1)
        beta = beta.view(beta.size(0), beta.size(1), 1, 1)      # (B, C, 1, 1)

        # Apply FiLM: scale and shift
        return gamma * x + beta


class ConditionalUNet(nn.Module):
    """
    Conditional UNet for polygon coloring.
    Takes an input image and color condition to generate colored polygon.
    """

    def __init__(self,
                 n_channels: int = 3,
                 n_classes: int = 3,
                 num_colors: int = 8,
                 color_embed_dim: int = 128,
                 bilinear: bool = True):
        super(ConditionalUNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.num_colors = num_colors
        self.color_embed_dim = color_embed_dim
        self.bilinear = bilinear

        # Color embedding layer
        self.color_embedding = nn.Sequential(
            nn.Linear(num_colors, color_embed_dim),
            nn.ReLU(inplace=True),
            nn.Linear(color_embed_dim, color_embed_dim),
            nn.ReLU(inplace=True)
        )

        # UNet encoder
        self.inc = DoubleConv(n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        factor = 2 if bilinear else 1
        self.down4 = Down(512, 1024 // factor)

        # FiLM layers for conditioning at different scales
        self.film1 = FiLM(color_embed_dim, 64)
        self.film2 = FiLM(color_embed_dim, 128)
        self.film3 = FiLM(color_embed_dim, 256)
        self.film4 = FiLM(color_embed_dim, 512)
        self.film5 = FiLM(color_embed_dim, 1024 // factor)

        # UNet decoder
        self.up1 = Up(1024, 512 // factor, bilinear)
        self.up2 = Up(512, 256 // factor, bilinear)
        self.up3 = Up(256, 128 // factor, bilinear)
        self.up4 = Up(128, 64, bilinear)
        self.outc = OutConv(64, n_classes)

        # Output activation
        self.output_activation = nn.Tanh()  # Output in [-1, 1] range

    def forward(self, x, color_condition):
        """
        Args:
            x: Input image tensor of shape (B, 3, H, W)
            color_condition: Color condition tensor of shape (B, num_colors) - one-hot encoded

        Returns:
            Generated colored polygon of shape (B, 3, H, W)
        """
        # Embed color condition
        color_embed = self.color_embedding(color_condition)  # (B, color_embed_dim)

        # Encoder path with FiLM conditioning
        x1 = self.inc(x)
        x1 = self.film1(x1, color_embed)

        x2 = self.down1(x1)
        x2 = self.film2(x2, color_embed)

        x3 = self.down2(x2)
        x3 = self.film3(x3, color_embed)

        x4 = self.down3(x3)
        x4 = self.film4(x4, color_embed)

        x5 = self.down4(x4)
        x5 = self.film5(x5, color_embed)

        # Decoder path
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)

        # Output layer
        logits = self.outc(x)
        output = self.output_activation(logits)

        return output


class AlternativeConditionalUNet(nn.Module):
    def __init__(self,
                 n_channels: int = 3,
                 n_classes: int = 3,
                 num_colors: int = 8,
                 bilinear: bool = True):
        super(AlternativeConditionalUNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.num_colors = num_colors
        self.bilinear = bilinear

        # Input channels = image channels + color channels
        input_channels = n_channels + num_colors

        # UNet architecture
        self.inc = DoubleConv(input_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        factor = 2 if bilinear else 1
        self.down4 = Down(512, 1024 // factor)

        self.up1 = Up(1024, 512 // factor, bilinear)
        self.up2 = Up(512, 256 // factor, bilinear)
        self.up3 = Up(256, 128 // factor, bilinear)
        self.up4 = Up(128, 64, bilinear)
        self.outc = OutConv(64, n_classes)

        self.output_activation = nn.Tanh()

    def forward(self, x, color_condition):
        """
        Args:
            x: Input image tensor of shape (B, 3, H, W)
            color_condition: Color condition tensor of shape (B, num_colors)

        Returns:
            Generated colored polygon of shape (B, 3, H, W)
        """
        batch_size, _, height, width = x.shape

        # Expand color condition to match spatial dimensions
        color_maps = color_condition.unsqueeze(-1).unsqueeze(-1)  # (B, num_colors, 1, 1)
        color_maps = color_maps.expand(-1, -1, height, width)     # (B, num_colors, H, W)

        # Concatenate image and color information
        x_conditioned = torch.cat([x, color_maps], dim=1)  # (B, 3+num_colors, H, W)

        # Standard UNet forward pass
        x1 = self.inc(x_conditioned)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)

        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)

        logits = self.outc(x)
        output = self.output_activation(logits)

        return output


def create_model(model_type: str = "film", **kwargs):
    if model_type == "film":
        return ConditionalUNet(**kwargs)
    elif model_type == "concat":
        return AlternativeConditionalUNet(**kwargs)
    else:
        raise ValueError(f"Unknown model type: {model_type}")


# Test the models
if __name__ == "__main__":
    # Test FiLM-based model
    model_film = ConditionalUNet(num_colors=8)

    # Test input
    batch_size = 2
    x = torch.randn(batch_size, 3, 256, 256)
    color_condition = torch.zeros(batch_size, 8)
    color_condition[0, 0] = 1  # First sample: color 0
    color_condition[1, 3] = 1  # Second sample: color 3

    # Forward pass
    output = model_film(x, color_condition)
    print(f"FiLM model output shape: {output.shape}")

    # Test concatenation-based model
    model_concat = AlternativeConditionalUNet(num_colors=8)
    output_concat = model_concat(x, color_condition)
    print(f"Concatenation model output shape: {output_concat.shape}")

    # Count parameters
    def count_parameters(model):
        return sum(p.numel() for p in model.parameters() if p.requires_grad)

    print(f"FiLM model parameters: {count_parameters(model_film):,}")
    print(f"Concatenation model parameters: {count_parameters(model_concat):,}")

FiLM model output shape: torch.Size([2, 3, 256, 256])
Concatenation model output shape: torch.Size([2, 3, 256, 256])
FiLM model parameters: 17,660,547
Concatenation model parameters: 17,267,715


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import wandb
import os
import time
from datetime import datetime
import numpy as np
from PIL import Image
import torchvision.transforms as transforms
from tqdm import tqdm
import matplotlib.pyplot as plt


class PolygonColoringTrainer:
    """
    Trainer class for the polygon coloring task using conditional UNet.
    """

    def __init__(self,
                 config: dict,
                 train_loader: DataLoader,
                 val_loader: DataLoader,
                 color_info: dict):

        self.config = config
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.color_info = color_info

        # Setup device
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(f"Using device: {self.device}")

        # Initialize model
        self.model = create_model(
            model_type=config['model_type'],
            num_colors=color_info['num_colors'],
            n_channels=config['n_channels'],
            n_classes=config['n_classes'],
            color_embed_dim=config.get('color_embed_dim', 128),
            bilinear=config.get('bilinear', True)
        ).to(self.device)

        # Count parameters
        num_params = sum(p.numel() for p in self.model.parameters() if p.requires_grad)
        print(f"Model has {num_params:,} trainable parameters")

        # Initialize optimizer
        if config['optimizer'] == 'adam':
            self.optimizer = optim.Adam(
                self.model.parameters(),
                lr=config['learning_rate'],
                weight_decay=config.get('weight_decay', 1e-4)
            )
        elif config['optimizer'] == 'adamw':
            self.optimizer = optim.AdamW(
                self.model.parameters(),
                lr=config['learning_rate'],
                weight_decay=config.get('weight_decay', 1e-2)
            )
        else:
            raise ValueError(f"Unsupported optimizer: {config['optimizer']}")

        # Initialize scheduler
        if config.get('scheduler') == 'cosine':
            self.scheduler = optim.lr_scheduler.CosineAnnealingLR(
                self.optimizer,
                T_max=config['epochs'],
                eta_min=config['learning_rate'] * 0.01
            )
        elif config.get('scheduler') == 'step':
            self.scheduler = optim.lr_scheduler.StepLR(
                self.optimizer,
                step_size=config.get('step_size', 50),
                gamma=config.get('gamma', 0.5)
            )
        else:
            self.scheduler = None

        # Initialize loss function
        if config['loss_function'] == 'mse':
            self.criterion = nn.MSELoss()
        elif config['loss_function'] == 'l1':
            self.criterion = nn.L1Loss()
        elif config['loss_function'] == 'huber':
            self.criterion = nn.SmoothL1Loss()
        else:
            raise ValueError(f"Unsupported loss function: {config['loss_function']}")

        # Training state
        self.current_epoch = 0
        self.best_val_loss = float('inf')
        self.train_losses = []
        self.val_losses = []

        # Create output directories
        os.makedirs(config['checkpoint_dir'], exist_ok=True)
        os.makedirs(config['sample_dir'], exist_ok=True)

    def train_epoch(self):
        self.model.train()
        epoch_loss = 0.0
        num_batches = len(self.train_loader)

        pbar = tqdm(self.train_loader, desc=f'Epoch {self.current_epoch + 1}/{self.config["epochs"]}')

        for batch_idx, batch in enumerate(pbar):
            # Move data to device
            input_images = batch['input_image'].to(self.device)
            color_conditions = batch['color_onehot'].to(self.device)
            target_images = batch['output_image'].to(self.device)

            # Forward pass
            self.optimizer.zero_grad()
            predicted_images = self.model(input_images, color_conditions)

            # Compute loss
            loss = self.criterion(predicted_images, target_images)

            # Backward pass
            loss.backward()

            # Gradient clipping (optional)
            if self.config.get('grad_clip'):
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.config['grad_clip'])

            self.optimizer.step()

            # Update metrics
            epoch_loss += loss.item()

            # Update progress bar
            pbar.set_postfix({'Loss': f'{loss.item():.4f}'})

            # Log to wandb
            if batch_idx % self.config.get('log_interval', 50) == 0:
                wandb.log({
                    'train_loss_step': loss.item(),
                    'learning_rate': self.optimizer.param_groups[0]['lr'],
                    'epoch': self.current_epoch,
                    'step': self.current_epoch * num_batches + batch_idx
                })

        avg_epoch_loss = epoch_loss / num_batches
        self.train_losses.append(avg_epoch_loss)

        return avg_epoch_loss

    def validate_epoch(self):
        self.model.eval()
        epoch_loss = 0.0
        num_batches = len(self.val_loader)

        # Store samples for visualization
        sample_inputs = []
        sample_predictions = []
        sample_targets = []
        sample_colors = []

        with torch.no_grad():
            for batch_idx, batch in enumerate(tqdm(self.val_loader, desc='Validating')):
                # Move data to device
                input_images = batch['input_image'].to(self.device)
                color_conditions = batch['color_onehot'].to(self.device)
                target_images = batch['output_image'].to(self.device)

                # Forward pass
                predicted_images = self.model(input_images, color_conditions)

                # Compute loss
                loss = self.criterion(predicted_images, target_images)
                epoch_loss += loss.item()

                # Store first batch samples for visualization
                if batch_idx == 0:
                    sample_inputs = input_images[:4].cpu()
                    sample_predictions = predicted_images[:4].cpu()
                    sample_targets = target_images[:4].cpu()
                    sample_colors = [batch['color_name'][i] for i in range(min(4, len(batch['color_name'])))]

        avg_epoch_loss = epoch_loss / num_batches
        self.val_losses.append(avg_epoch_loss)

        # Create and log sample images
        if len(sample_inputs) > 0:
            self.log_sample_images(
                sample_inputs, sample_predictions, sample_targets, sample_colors
            )

        return avg_epoch_loss

    def log_sample_images(self, inputs, predictions, targets, colors):
        # Denormalize images from [-1, 1] to [0, 1]
        def denormalize(tensor):
            return (tensor + 1) / 2

        inputs = denormalize(inputs)
        predictions = denormalize(predictions)
        targets = denormalize(targets)

        # Create comparison grid
        fig, axes = plt.subplots(3, 4, figsize=(16, 12))

        for i in range(4):
            # Input image
            axes[0, i].imshow(inputs[i].permute(1, 2, 0))
            axes[0, i].set_title(f'Input ({colors[i]})')
            axes[0, i].axis('off')

            # Predicted image
            axes[1, i].imshow(predictions[i].permute(1, 2, 0))
            axes[1, i].set_title('Predicted')
            axes[1, i].axis('off')

            # Target image
            axes[2, i].imshow(targets[i].permute(1, 2, 0))
            axes[2, i].set_title('Target')
            axes[2, i].axis('off')

        plt.tight_layout()

        # Log to wandb
        wandb.log({
            'sample_images': wandb.Image(fig),
            'epoch': self.current_epoch
        })

        plt.close(fig)

        # Save sample images to disk
        sample_path = os.path.join(
            self.config['sample_dir'],
            f'epoch_{self.current_epoch:03d}.png'
        )

        # Create a single image with all samples
        grid_img = torch.cat([
            torch.cat([inputs[i] for i in range(4)], dim=2),
            torch.cat([predictions[i] for i in range(4)], dim=2),
            torch.cat([targets[i] for i in range(4)], dim=2)
        ], dim=1)

        transforms.ToPILImage()(grid_img).save(sample_path)

    def save_checkpoint(self, is_best=False):
        checkpoint = {
            'epoch': self.current_epoch,
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'scheduler_state_dict': self.scheduler.state_dict() if self.scheduler else None,
            'train_losses': self.train_losses,
            'val_losses': self.val_losses,
            'best_val_loss': self.best_val_loss,
            'config': self.config,
            'color_info': self.color_info
        }

        # Save latest checkpoint
        latest_path = os.path.join(self.config['checkpoint_dir'], 'latest_checkpoint.pth')
        torch.save(checkpoint, latest_path)

        # Save best checkpoint
        if is_best:
            best_path = os.path.join(self.config['checkpoint_dir'], 'best_checkpoint.pth')
            torch.save(checkpoint, best_path)
            print(f"New best model saved with validation loss: {self.best_val_loss:.4f}")

    def load_checkpoint(self, checkpoint_path):
        checkpoint = torch.load(checkpoint_path, map_location=self.device)

        self.model.load_state_dict(checkpoint['model_state_dict'])
        self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

        if self.scheduler and checkpoint['scheduler_state_dict']:
            self.scheduler.load_state_dict(checkpoint['scheduler_state_dict'])

        self.current_epoch = checkpoint['epoch']
        self.train_losses = checkpoint['train_losses']
        self.val_losses = checkpoint['val_losses']
        self.best_val_loss = checkpoint['best_val_loss']

        print(f"Loaded checkpoint from epoch {self.current_epoch}")

    def train(self):
        print("Starting training...")
        start_time = time.time()

        for epoch in range(self.current_epoch, self.config['epochs']):
            self.current_epoch = epoch

            # Train
            train_loss = self.train_epoch()

            # Validate
            val_loss = self.validate_epoch()

            # Update scheduler
            if self.scheduler:
                self.scheduler.step()

            # Log epoch metrics
            wandb.log({
                'epoch': epoch,
                'train_loss_epoch': train_loss,
                'val_loss_epoch': val_loss,
                'learning_rate': self.optimizer.param_groups[0]['lr']
            })

            # Save checkpoint
            is_best = val_loss < self.best_val_loss
            if is_best:
                self.best_val_loss = val_loss

            if epoch % self.config.get('save_interval', 10) == 0 or is_best:
                self.save_checkpoint(is_best=is_best)

            # Print epoch summary
            elapsed_time = time.time() - start_time
            print(f"Epoch {epoch + 1}/{self.config['epochs']} - "
                  f"Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, "
                  f"Time: {elapsed_time/60:.1f}min")

        print(f"Training completed! Best validation loss: {self.best_val_loss:.4f}")


def main():
    # Training configuration
    config = {
        # Model configuration
        'model_type': 'film',  # 'film' or 'concat'
        'n_channels': 3,
        'n_classes': 3,
        'color_embed_dim': 128,
        'bilinear': True,

        # Training configuration
        'epochs': 100,
        'batch_size': 16,
        'learning_rate': 1e-3,
        'optimizer': 'adamw',
        'weight_decay': 1e-2,
        'scheduler': 'cosine',
        'loss_function': 'l1',  # 'mse', 'l1', 'huber'
        'grad_clip': 1.0,

        # Data configuration
        'image_size': 256,
        'num_workers': 4,
        'use_augmentations': True,

        # Logging and saving
        'log_interval': 20,
        'save_interval': 10,
        'checkpoint_dir': 'checkpoints',
        'sample_dir': 'samples',

        # Paths
        'train_json': '/content/drive/MyDrive/dataset/dataset/training/data.json',
        'val_json': '/content/drive/MyDrive/dataset/dataset/validation/data.json',
        'train_input_dir': '/content/drive/MyDrive/dataset/dataset/training/inputs',
        'train_output_dir': '/content/drive/MyDrive/dataset/dataset/training/outputs',
        'val_input_dir': '/content/drive/MyDrive/dataset/dataset/validation/inputs',
        'val_output_dir': '/content/drive/MyDrive/dataset/dataset/validation/outputs',
    }

    # Initialize wandb
    wandb.init(
        project="polygon-coloring",
        config=config,
        name=f"unet-{config['model_type']}-{datetime.now().strftime('%Y%m%d-%H%M%S')}"
    )


    # Create data loaders
    train_loader, val_loader, color_info = get_data_loaders(
        train_json_path=config['train_json'],
        val_json_path=config['val_json'],
        train_input_dir=config['train_input_dir'],
        train_output_dir=config['train_output_dir'],
        val_input_dir=config['val_input_dir'],
        val_output_dir=config['val_output_dir'],
        batch_size=config['batch_size'],
        image_size=config['image_size'],
        num_workers=config['num_workers'],
        use_augmentations=config['use_augmentations']
    )

    # Initialize trainer
    trainer = PolygonColoringTrainer(config, train_loader, val_loader, color_info)

    # Start training
    trainer.train()

    # Finish wandb run
    wandb.finish()


if __name__ == "__main__":
    main()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: meghashyam2005 (meghashyam2005-mahindra-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Dataset initialized with 56 samples
Colors: ['blue', 'cyan', 'green', 'magenta', 'orange', 'purple', 'red', 'yellow']
Color mapping: {'blue': 0, 'cyan': 1, 'green': 2, 'magenta': 3, 'orange': 4, 'purple': 5, 'red': 6, 'yellow': 7}
Dataset initialized with 5 samples
Colors: ['blue', 'cyan', 'green', 'magenta', 'orange', 'purple', 'red', 'yellow']
Color mapping: {'blue': 0, 'cyan': 1, 'green': 2, 'magenta': 3, 'orange': 4, 'purple': 5, 'red': 6, 'yellow': 7}
Using device: cuda


...

Validating: 100%|██████████| 1/1 [00:00<00:00,  1.46it/s]


Epoch 99/100 - Train Loss: 0.0434, Val Loss: 0.1164, Time: 11.1min


Validating: 100%|██████████| 1/1 [00:00<00:00,  1.55it/s]


Epoch 100/100 - Train Loss: 0.0445, Val Loss: 0.1159, Time: 11.2min
Training completed! Best validation loss: 0.1094


epoch,▁▁▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▇▇▇▇▇▇████
learning_rate,█████▇▇▇▇▇▇▇▇▇▇▆▆▆▆▅▄▄▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁
step,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇███
train_loss_epoch,█▆▄▄▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss_step,█▅▅▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss_epoch,▆▇█▇▅▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,99
learning_rate,1e-05
step,297
train_loss_epoch,0.04452
train_loss_step,0.04889


In [ ]:
import os
from google.colab import files

checkpoint_dir = 'checkpoints'
checkpoint_path = os.path.join(checkpoint_dir, 'best_checkpoint.pth')

if os.path.exists(checkpoint_path):
    print(f"Downloading {checkpoint_path}...")
    files.download(checkpoint_path)
else:
    print(f"Best checkpoint not found at {checkpoint_path}. Please ensure training completed successfully and the file exists.")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>